![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll provide the database design script as a `.sql` file that they can then run. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description |
|--------|-----------|-------------|
| `id` | `serial` | Client ID - primary key |
| `age` | `integer` | Client's age in years |
| `job` | `text` | Client's type of job |
| `marital` | `text` | Client's marital status |
| `education` | `text` | Client's level of education |
| `credit_default` | `boolean` | Whether the client's credit is in default |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) |
| `loan` | `boolean` | Whether the client has an existing personal loan |

<br>

## campaign

| column | data type | description |
|--------|-----------|-------------|
| `campaign_id` | `serial` | Campaign ID - primary key |
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign |
| `contact_duration` | `integer` | Last contact duration in seconds |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign |
| `previous_outcome` | `boolean` | Outcome of the previous campaign |
| `campaign_outcome` | `boolean` | Outcome of the current campaign |
| `last_contact_date` | `date` | Last date the client was contacted |

<br>

## economics

| column | data type | description |
|--------|-----------|-------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) |
| `number_employed` | `float` | Number of employees (quarterly indicator)| 

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('bank_marketing.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   client_id       41188 non-null  int64  
 1   age             41188 non-null  int64  
 2   job             41188 non-null  object 
 3   marital         41188 non-null  object 
 4   education       41188 non-null  object 
 5   credit_default  41188 non-null  object 
 6   housing         41188 non-null  object 
 7   loan            41188 non-null  object 
 8   contact         41188 non-null  object 
 9   month           41188 non-null  object 
 10  day             41188 non-null  int64  
 11  duration        41188 non-null  int64  
 12  campaign        41188 non-null  int64  
 13  pdays           41188 non-null  int64  
 14  previous        41188 non-null  int64  
 15  poutcome        41188 non-null  object 
 16  emp_var_rate    41188 non-null  float64
 17  cons_price_idx  41188 non-null 

### Create DataFrame `client_df`

In [4]:
client_df = data[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan']]
client_df.rename(columns={'client_id': 'id'}, inplace=True)

### Create DataFrame `campaign_df`

In [5]:
campaign_df = data[['client_id', 'campaign', 'duration', 'pdays', 'previous', 'poutcome',
                    'y', 'month', 'day']]
campaign_df.rename(columns={'campaign': 'number_contacts', 'duration': 'contact_duration', 'previous': 'previous_campaign_contacts', 'poutcome':'previous_outcome', 'y':'campaign_outcome'}, inplace=True)

### Create DataFrame `economics_df`

In [6]:
economics_df = data[['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed']]
economics_df.rename(columns={'euribor3m': 'euribor_three_months', 'nr_employed': 'number_employed'}, inplace=True)

### Cleaning the `client_df`

In [7]:
client_df['education'] = client_df['education'].str.replace('.','_')

In [8]:
client_df = client_df.replace({'education' : {'unknown':np.nan}})

In [9]:
client_df['job'] = client_df['job'].str.replace('.','')

### Cleaning the `campaign_df`

In [10]:
campaign_df = campaign_df.replace({'previous_outcome' : {'success':'1', 'failure':'2', 'nonexistent':np.nan}})

In [11]:
campaign_df['campaign_id'] = 1

In [12]:
campaign_df['year'] = 2022

In [13]:
campaign_df['month'].unique()

array(['may', 'jun', 'jul', 'aug', 'oct', 'nov', 'dec', 'mar', 'apr',
       'sep'], dtype=object)

In [14]:
month_dict = {'may':5, 'jun':6, 'jul':7, 'aug':8, 'oct':10, 'nov':11, 'dec':12, 'mar':3, 'apr':4, 'sep':9}

In [15]:
campaign_df = campaign_df.replace({'month' : month_dict})

In [16]:
campaign_df['last_contact_date'] = pd.to_datetime(campaign_df[['year', 'month', 'day',]], 
                                                  format='%Y%m%d').dt.strftime('%Y-%m-%d')

In [17]:
campaign_df.drop(columns=['year', 'day', 'month'], inplace=True)

In [18]:
campaign_df = campaign_df[['campaign_id', 'client_id', 'number_contacts', 'contact_duration', 'pdays', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome', 'last_contact_date']]

### Save csv files

In [19]:
client_df.to_csv('client.csv', index=False)
campaign_df.to_csv('campaign.csv', index=False)
economics_df.to_csv('economics.csv', index=False)

### Create SQL tables

In [20]:
client_table = '''
CREATE TABLE client
(id SERIAL PRIMARY KEY,
age INTEGER,
job TEXT, 
marital TEXT,
education TEXT,
credit_default BOOLEAN, 
housing BOOLEAN,
loan BOOLEAN);

\copy client from 'client.csv' DELIMITER ',' CSV HEADER
'''

In [21]:
campaign_table = '''
CREATE TABLE campaign
(
campaign_id SERIAL PRIMARY KEY,
client_id SERIAL REFERENCES client (id),
number_contacts INTEGER, 
contact_duration INTEGER,
pdays INTEGER,
previous_campaign_contacts INTEGER, 
previous_outcome BOOLEAN,
campaign_outcome BOOLEAN,
last_contact_date DATE
);

\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
'''

In [22]:
economics_table = '''
CREATE TABLE economics
(
client_id SERIAL REFERENCES client (id),
emp_var_rate FLOAT,
cons_price_idx FLOAT, 
euribor_three_months FLOAT,
number_employed FLOAT
);

\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
'''